In [1]:
from PIL import Image
import os
import sys
import numpy as np
import time
from sklearn import svm
import joblib
import warnings
warnings.filterwarnings('ignore')

import gzip
import struct

In [5]:
# 假设 gz 文件存储路径
def get_data_from_gz(gz_file_path):
    with gzip.open(gz_file_path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8)
    return data

# 从 .gz 文件中加载图像和标签
def load_gz_data(image_gz_path, label_gz_path):
    # 载入图像数据
    images = get_data_from_gz(image_gz_path)
    labels = get_data_from_gz(label_gz_path)
    
    # 假设图像大小为 16x16，转换为图像矩阵 (num_samples, 16, 16)
    num_samples = len(labels)
    images = images.reshape(num_samples, 16, 16)
    return images, labels

# 将 16x16 图像转换为 1x256 向量
def img2vector(img):
    img_normalized = np.round(img / 255)
    img_vector = img_normalized.reshape(1, -1)
    return img_vector

# 准备数据，将图像展平成向量
def prepare_data(images):
    num_samples = images.shape[0]
    dataMat = np.zeros((num_samples, 256))  # 存储每个图像的 1x256 向量
    for i in range(num_samples):
        img_vector = img2vector(images[i])
        dataMat[i, :] = img_vector
    return dataMat

# 训练模型并保存
def train_svm(dataMat, dataLabel, model_path, decision='ovr'):
    clf = svm.SVC(decision_function_shape=decision)
    clf.fit(dataMat, dataLabel)
    joblib.dump(clf, model_path)
    return clf

# 预测测试集
def evaluate_model(clf, test_dataMat, test_dataLabel):
    start_time = time.time()
    accuracy = clf.score(test_dataMat, test_dataLabel)
    end_time = time.time()
    print(f"准确率: {accuracy:.6f}")
    print(f"错误率: {1 - accuracy:.6f}")
    print(f"预测时间: {end_time - start_time:.6f} 秒")


In [6]:
# 假设有训练集和测试集的 .gz 文件
train_image_gz = r'E:\ipynb\project\cv_exp_mnist\train-images-idx3-ubyte.gz'
train_label_gz = r'E:\ipynb\project\cv_exp_mnist\train-labels-idx1-ubyte.gz'
test_image_gz = r'E:\ipynb\project\cv_exp_mnist\t10k-images-idx3-ubyte.gz'
test_label_gz = r'E:\ipynb\project\cv_exp_mnist\t10k-labels-idx1-ubyte.gz'
    
# 加载训练数据
train_images, train_labels = load_gz_data(train_image_gz, train_label_gz)
train_dataMat = prepare_data(train_images)
    
# 加载测试数据
test_images, test_labels = load_gz_data(test_image_gz, test_label_gz)
test_dataMat = prepare_data(test_images)

ValueError: cannot reshape array of size 47040016 into shape (60008,16,16)

In [4]:
# 训练并保存模型
model_path = 'svm.model'
clf = train_svm(train_dataMat, train_labels, model_path)

NameError: name 'train_dataMat' is not defined

In [ ]:
# 加载模型并评估
clf = joblib.load(model_path)
evaluate_model(clf, test_dataMat, test_labels)